In [1]:
import sys
sys.path.append('..')
import crystalmaths
from crystalmaths.find_matching_angles import find_matching_angles
import itertools
import requests
import numpy as np
import pandas as pd
%matplotlib qt
# from crystalmaths import 


In [4]:
image_filepath = r'C:\Users\lacho\OneDrive - UW\crystalmaths\Local_data\SL_Bornite_011.png'

image_object = crystalmaths.imagetools.ImageHandler(image_filepath=image_filepath)
image_object.show_image()

In [5]:
image_object.image_array.shape

image_object.get_scale_ratio()
image_object.scale_ratio

Enter scalebar length in nanometers.2
length_nm =  2.0
length_pixel = 22.740857468890937


11.370428734445468

In [6]:
image_object.get_planes()

Pick planes using left mouse button. Right click once done. Middle mouse button removes most recent point.
Exited zoom loop
Entered point pick loop
Saved points =  [(238.56765201325877, 109.53205999021895), (165.29155572461016, 72.47288485573)]
Press keyboard button to save points, mouseclick to restart


In [7]:
image_object.point_coordinates 

[[83.0, -45.0], [10.0, -82.0]]

In [12]:
def get_zone_axis(image_object, d_spacing_tolerance, angle_tolerance):
    """
    get_zone_axis will take one image_object which has had two planes
    selected on it. The two points (e.g. d1, d2) will be passed as 
    arguments to get_d, which searches the ACMS online database to
    generate a list of candidate minerals, based on the d-spacing 
    of the selected points from fft image.
    
    Tolerances for search criteria are also passed as float arguments into
    this function to provide cutoffs for difference in d-spacing and in 
    angles. 
    
    get_zone_axis returns a DataFrame with the HKL and mineral name 
    for all sets of matching planes
    """
    angle_compare_list=[]
    p1 = image_object.point_coordinates[0]
    p2 = image_object.point_coordinates[1]
    fft_angle = crystalmaths.angles_from_fft.angle(p1, p2)
    d1 = crystalmaths.distance_from_fft.distance_from_fft(p1, image_object.scale_ratio, image_object.image_array.shape[0])
    d2 = crystalmaths.distance_from_fft.distance_from_fft(p2, image_object.scale_ratio, image_object.image_array.shape[0])
    link = crystalmaths.get_d.make_web_address(d1, d2, tolerance=d_spacing_tolerance)
    print(link)
    link_list = crystalmaths.get_d.compile_links(link)
    query_results = crystalmaths.get_d.get_d(link_list, [d1, d2])
    print(d1, d2)
    for i,result in enumerate(query_results):
        data_df = result[0]
        metadata_df = result[1]
        temp_object = crystalmaths.angles_from_structure_data.AllAnglePairs(data_df, metadata_df)
        result_df = temp_object.result_df
        result_df = find_matching_angles(fft_angle, result_df, angle_tolerance)
        final_df = result_df[result_df['angle match']==True]
#         df[df['Btu']>1500000]
        if final_df.empty:
            pass
        else:
            mineral_name = metadata_df['Mineral_Name'].values
            n = final_df.shape[0]
            mineral_name_list = [mineral_name for i in range(n)]
            final_df.insert(0, 'Mineral_Name', mineral_name_list)
            angle_compare_list.append((final_df, metadata_df))
    return angle_compare_list

awesome_list = get_zone_axis(image_object, 0.001, 10)

http://rruff.geo.arizona.edu/AMS/result.php?diff=vals(2.8876764989388146,3.3003906205171347),opt(),type(d-spacing),tolerance(0.001)
2.8876764989388146 3.3003906205171347
0.07213592030928428 0.09890495874118392
0.014691346525840929 0.09965438822873852
0.07681483547871144 0.10169758082021751
0.09596302673985924 0.10413701517169983
0.07257619108259108 0.09922652557045748
0.05876538610336375 0.09997839165641065
0.09899861219190845 0.10202822722081635
0.10360221385165089 0.10447559283458938
0.058325115330056944 0.0994189664234057
0.07345673262920469 0.10017229068548718
0.10159392687065086 0.10222610172018322
0.09655005443760162 0.10467821377774719
0.08638699606181843 0.10000497826371393
-6.169938527995982e-18 0.10076274289520432
0.02879566535393947 0.10282865984517371
0.057591330707878946 0.1052952255502779
0.08726753760843203 0.10076659024289926
0.08814807915504563 0.10153012581330456
0.0731632187803335 0.10361177625099043
0.07286970493146229 0.10609712668082602
0.07948159357220476 0.10207

0.08979777540977717 0.1061818360672136
-0.022662198255358736 0.1078370154184368
0.0986308356580294 0.106781561375403
0.05136375545295567 0.10717225690514674
0.07123225240230037 0.10849458148210005
0.07973368055586282 0.10893916457276479
-0.042080595398358955 0.10628707103647629
0.09114889478905192 0.10524678706336767
0.09654854397880176 0.10563186711554061
0.10476648547508124 0.10693518588506107
0.03815053329953854 0.10737337897076306
0.09227069357761417 0.10618748466536106
0.0238585900057091 0.10514817539317137
-0.05139839830826741 0.10553289464205365
0.01051419466025882 0.10683499225847055
0.09184518477178528 0.10727277477627538
0.07285229643461394 0.1071306965824471
-8.715190935198182e-18 0.1051075830996474
-0.02266219825535874 0.1078370154184368
-0.04208059539835896 0.10628707103647629
0.09227069357761417 0.10618748466536106
0.0388260929891759 0.10608215563013845
0.08979777540977717 0.10407884335656782
0.0986308356580294 0.106781561375403
0.09114889478905192 0.10524678706336767
0.0

In [11]:
for thing in awesome_list:
    print(thing[0]['Mineral_Name'], thing[0]['zone axis'])



5     [Dumortierite]
8     [Dumortierite]
15    [Dumortierite]
Name: Mineral_Name, dtype: object 5      [-6.0, 0.0, 12.0]
8     [-3.0, 3.0, -12.0]
15     [0.0, -4.0, 12.0]
Name: zone axis, dtype: object
5     [Dumortierite]
8     [Dumortierite]
15    [Dumortierite]
Name: Mineral_Name, dtype: object 5      [12.0, -6.0, 0.0]
8     [-12.0, -3.0, 3.0]
15     [12.0, 0.0, -4.0]
Name: zone axis, dtype: object
4     [Nb3 O15 P3 Rb]
15    [Nb3 O15 P3 Rb]
26    [Nb3 O15 P3 Rb]
Name: Mineral_Name, dtype: object 4      [-4.0, 2.0, 14.0]
15    [-1.0, 4.0, -14.0]
26    [5.0, -1.0, -13.0]
Name: zone axis, dtype: object
3    [Ammonioborite]
8    [Ammonioborite]
Name: Mineral_Name, dtype: object 3    [4.0, -10.0, 18.0]
8    [-8.0, -16.0, 0.0]
Name: zone axis, dtype: object


In [ ]:
for i,result in enumerate(query_results):
    data_df = result[0]
    metadata_df = result[1]
    temp_object = crystalmaths.angles_from_structure_data.AllAnglePairs(data_df, metadata_df)
    new_tuple_list.append(tuple(temp_object.result_df, metadata_df))
# temp_object.result_df.head()

In [ ]:
fake_angle = [temp_object.result_df['angle'].mean()]
result_df = temp_object.result_df
result_df = find_matching_angles(fake_angle, result_df, 2)
result_df

Future Work

# Edwin's code starts from here.

In [ ]:
#This is the function passing all data frame of query results
#Start of all testing and comparison
full_angle_list = []
full_plane1_list = []
full_plane2_list = []
for k in range(len(query_results)):
    plane1_list = []
    plane2_list = []
    df1 = query_results[k][0]
    df11 = df1.loc[df1['D-REF'] == 3.3]
    df12 = df1.loc[df1['D-REF'] == 3.5]
    df_p = query_results[k][1]
    a = []
    lattice_parameter = [float(df_p['a'][0]), float(df_p['b'][0]), float(df_p['c'][0])]
    alpha = float(df_p['alpha'][0])
    beta = float(df_p['beta'][0])
    gamma = float(df_p['gamma'][0])
    for i in range(df11.shape[0]):
        for j in range(df12.shape[0]):
            plane1 = [df11['H'].iloc[i], df11['K'].iloc[i], df11['L'].iloc[i]]
            plane2 = [df12['H'].iloc[j], df12['K'].iloc[j], df12['L'].iloc[j]]
            a.append(crystalmaths.temp_angles_from_structure_data_Edwin.angle_for_all(plane1, plane2, lattice_parameter, alpha, beta, gamma))
            plane1_list.append(plane1)
            plane2_list.append(plane2)
#     print(a)
    full_angle_list.append(a)
    full_plane1_list.append(plane1_list)
    full_plane2_list.append(plane2_list)
    
full_angle_list

In [ ]:
#Given angles and find zone axis
angle_list1 = [11, 19, 34, 42]
tolerance = 1
matching_angle_list = []
for i in range(5):
    angle_list2 = full_angle_list[i]
    c = crystalmaths.find_matching_angles.find_matching_angles(angle_list1, angle_list2, tolerance)
    matching_angle_list.append(c)
    
interested_plane1 = []
interested_plane2 = []
zone_axis_dic = {}
for j in range(len(matching_angle_list)):
    zone_axis = []
    for k in range(len(matching_angle_list[j])):
        for l in range(len(full_angle_list[j])):
            if matching_angle_list[j][k][1] == full_angle_list[j][l]:
                interested_plane1.append(full_plane1_list[j][l])
                interested_plane2.append(full_plane2_list[j][l])
                zone_axis.append(np.cross(full_plane1_list[j][l], full_plane2_list[j][l]))  
                zone_axis_dic[query_results[j][1]['Mineral_Name'][0]] = zone_axis
print(zone_axis_dic)

            
            
